Implement your own index that would replace the one from elasticsearch used in HW1, then index the document collection used in HW1. Your index should be able to handle large numbers of documents and terms without using excessive memory or disk I/O.

# Setup

In [24]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import os
import pandas as pd
import re
import string
import json
from collections import OrderedDict
from pandarallel import pandarallel
from tqdm import tqdm
import math
import numpy as np
import concurrent.futures
import zlib

pandarallel.initialize(progress_bar=True)

# Path to the directory containing files
DIRECTORY_PATH = '../ap89_collection/'
QUERY_PATH = '../AP_DATA/query_desc.51-100.short.txt'
INDEX_NAME = 'ap89_collection_hw2'
STOP_WORDS_PATH = '../AP_DATA/stoplist.txt'
BATCH_SIZE = 1000
STEMMED_PATH = 'stemmed/'
UNSTEMMED_PATH = 'unstemmed/'

STEMMED_CATALOG_PATH = os.path.join(STEMMED_PATH, 'catalog/')
UNSTEMMED_CATALOG_PATH = os.path.join(UNSTEMMED_PATH, 'catalog/')
STEMMED_INDEX_PATH = os.path.join(STEMMED_PATH, 'index/')
UNSTEMMED_INDEX_PATH = os.path.join(UNSTEMMED_PATH, 'index/')
STEMMED_MERGE_PATH = os.path.join(STEMMED_PATH, 'merge/')
UNSTEMMED_MERGE_PATH = os.path.join(UNSTEMMED_PATH, 'merge/')
STEMMED_RESULTS_PATH = os.path.join(STEMMED_PATH, 'results/')
UNSTEMMED_RESULTS_PATH = os.path.join(UNSTEMMED_PATH, 'results/')
STEMMED_COMPRESSED_PATH = os.path.join(STEMMED_PATH, 'compressed/')
UNSTEMMED_COMPRESSED_PATH = os.path.join(UNSTEMMED_PATH, 'compressed/')
STEMMED_DECOMPRESSED_PATH = os.path.join(STEMMED_PATH, 'decompressed/')
UNSTEMMED_DECOMPRESSED_PATH = os.path.join(UNSTEMMED_PATH, 'decompressed/')

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Task 1 - A tokenizer

In [2]:
def parse_documents(path=DIRECTORY_PATH, stop_words_path=STOP_WORDS_PATH):
    data = []
    
    stop_words = set()
    with open(stop_words_path, 'r') as file:
        stop_words = set(file.read().splitlines())

    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)

        with open(file_path, 'r', encoding='ISO-8859-1', errors='ignore') as file:
            content = file.read()

        # Use regular expression to extract each document
        docs = re.findall(r'<DOC>.*?</DOC>', content, flags=re.DOTALL)

        # Extract metadata and text for each document
        for doc in docs:
            docno_match = re.search(r'<DOCNO>(.*?)</DOCNO>', doc)
            fileid_match = re.search(r'<FILEID>(.*?)</FILEID>', doc)
            first_match = re.search(r'<FIRST>(.*?)</FIRST>', doc)
            second_match = re.search(r'<SECOND>(.*?)</SECOND>', doc)
            head_matches = re.findall(r'<HEAD>(.*?)</HEAD>', doc)
            byline_match = re.search(r'<BYLINE>(.*?)</BYLINE>', doc)
            dateline_match = re.search(r'<DATELINE>(.*?)</DATELINE>', doc)
            text_match = re.search(r'<TEXT>(.*?)</TEXT>', doc, flags=re.DOTALL)

            # Create a dictionary for each document
            doc_data = {
                'DOCNO': docno_match.group(1).strip() if docno_match else None,
                'FILEID': fileid_match.group(1).strip() if fileid_match else None,
                'FIRST': first_match.group(1).strip() if first_match else None,
                'SECOND': second_match.group(1).strip() if second_match else None,
                'HEAD': head_matches if head_matches else None,
                'BYLINE': byline_match.group(1).strip() if byline_match else None,
                'DATELINE': dateline_match.group(1).strip() if dateline_match else None,
                'TEXT': text_match.group(1).strip() if text_match else None
            }
            data.append(doc_data)

    df = pd.DataFrame(data)
    df['TEXT'] = df['TEXT'].replace(r'\n', ' ', regex=True)
    df['STEMMED_TEXT'] = df['TEXT'].parallel_apply(lambda x: preprocess_text(x, stop_words, stem=True))
    df['UNSTEMMED_TEXT'] = df['TEXT'].parallel_apply(lambda x: preprocess_text(x, stop_words, stem=False))

    return df

def preprocess_text(text, stop_words, stem=True):
    # Tokenize the words and then remove the stop words
    # then use porter stemmer to stem the words
    tokens = word_tokenize(text)
    punctuations = set(string.punctuation)
    # special_exclusions = ['``', "''", '..', '...', '....', '--']
    # punctuations.update(special_exclusions)
    # Remove words from stop words and punctuation
    tokens = [word.lower() for word in tokens if word.lower() not in stop_words and word not in punctuations]
    # Remove if same puncatuations are repeated
    for word in tokens:
        if all(c in string.punctuation for c in word):
            tokens.remove(word)
    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
        
    return ' '.join(tokens)

df = parse_documents()
df.to_csv('ap89_processed_collection.csv', index=False)
df.head()

,DOCNO,FILEID,FIRST,SECOND,HEAD,BYLINE,DATELINE,TEXT,STEMMED_TEXT,UNSTEMMED_TEXT
0,AP891220-0001,AP-NR-12-20-89 2343EST,u i BC-Panama-Resistance 12-20 0723,"BC-Panama-Resistance,0746","[Defense Forces, Noriega Resist; Loyalty Remai...",By ELOY O. AGUILAR,"PANAMA CITY, Panama (AP)",Instead of collapsing when the United States t...,collaps unit state threw militari gen. manuel ...,collapsing united states threw military gen. m...
1,AP891220-0002,AP-NR-12-20-89 0031EST,r a AM-RollingStones 12-20 0372,"AM-Rolling Stones,0385",[Stones Pay Per View Concert Tops North Americ...,By HENRY STERN,"ATLANTIC CITY, N.J. (AP)",The Rolling Stones capped the next-to-last sho...,roll stone cap next-to-last show 32-citi north...,rolling stones capped next-to-last show 32-cit...
2,AP891220-0003,AP-NR-12-20-89 0038EST,r a PM-Christmas-Jesus'KinI 12-20 1240,"PM-Christmas-Jesus' Kin I,1277",[Part I: Jesus Descended From Checkered Ancest...,By GEORGE W. CORNELL,None,This first installment of a three-part Christm...,instal three-part christma seri rel jesu deal ...,installment three-part christmas series relati...
3,AP891220-0004,AP-NR-12-20-89 0039EST,r i PM-SAfrica-CoupTrial 12-20 0176,"PM-SAfrica-Coup Trial,0178",[Leader Of Failed Homeland Coup Gets 18-Year P...,None,"JOHANNESBURG, South Africa (AP)",A soldier who led a failed coup attempt in one...,soldier led fail coup attempt south africa 's ...,soldier led failed coup attempt south africa '...
4,AP891220-0005,AP-NR-12-20-89 0039EST,r a PM-RobinsonProfile 12-20 0558,"PM-Robinson Profile,0576","[Savannah Leaders Praise Slain Alderman, Eds: ...",By LAURAN NEERGAARD,"SAVANNAH, Ga. (AP)","Robert Robinson, the alderman and lawyer assas...",robert robinson alderman lawyer assassin mail ...,robert robinson alderman lawyer assassinated m...


# Task 2 - indexer

In [3]:
def indexer(subset_df, tokens_dict, stem=True):
    doc_nos = subset_df.index
    if stem:
        tokenized_texts = subset_df['STEMMED_TEXT'].apply(lambda x: x.split())
    else:
        tokenized_texts = subset_df['UNSTEMMED_TEXT'].apply(lambda x: x.split())
    
    for doc_no, tokenized_text in zip(doc_nos, tokenized_texts):
        for i,token in enumerate(tokenized_text, start=1):
            if stem:
                token = STEMMED_TOKEN_TO_WORD[token]
            else:
                token = UNSTEMMED_TOKEN_TO_WORD[token]
            if token not in tokens_dict:
                tokens_dict[token] = {}
            if doc_no not in tokens_dict[token]:
                tokens_dict[token][doc_no] = []
            tokens_dict[token][doc_no].append(i)

def write_index_to_file(tokens_dict, batch_number, stem=True):
    if stem:
        catalogs_path = STEMMED_CATALOG_PATH
        indices_path = STEMMED_INDEX_PATH
    else:
        catalogs_path = UNSTEMMED_CATALOG_PATH
        indices_path = UNSTEMMED_INDEX_PATH
    catalog_name = f'{catalogs_path}catalog_{batch_number}.txt'
    inverted_index_name = f'{indices_path}inverted_index_{batch_number}.txt'
    with open(catalog_name, 'w') as catalog_file , open(inverted_index_name, 'w') as inverted_index:

        # Catalog should contain term, offset, and length
        offset = 0
        for token, docs in tokens_dict.items():
            index_string = []
            for doc, pos_list in docs.items():
                pos_string = ",".join(str(pos).strip() for pos in pos_list)
                index_string.append(f"{doc}:{pos_string}|")
            index_string = "".join(index_string)
            inverted_index.write(index_string)
            catalog_file.write(f"{token}:{offset},{len(index_string)}\n")
            offset += len(index_string)            
        
def process_batches(dataframe, batch_size, stem = True):
    for i in range(0, len(dataframe), batch_size):
        print(f'Processing batch {(i+1)//batch_size}/ {len(dataframe)//batch_size}') 
        tokens_dict = {}
        batch = dataframe[i:i + batch_size]
        indexer(batch, tokens_dict, stem)
        tokens_dict = sort_tokens(tokens_dict)
        write_index_to_file(tokens_dict, i, stem)

def sort_tokens(tokens):
    return OrderedDict(sorted(tokens.items(), key=lambda t: t[0]))

# Map doc number to df index
DOC_NO_MAP = {i: doc_no for i, doc_no in enumerate(df['DOCNO'])}
# Create doc len map for both stemmed and unstemmed
DOC_LEN_MAP_STEM = {i: len(doc.split()) for i, doc in enumerate(df['STEMMED_TEXT'])}
DOC_LEN_MAP_UNSTEM = {i: len(doc.split()) for i, doc in enumerate(df['UNSTEMMED_TEXT'])}

STEMMED_TOKENS = set()
UNSTEMMED_TOKENS = set()
for doc in df['STEMMED_TEXT']:
    STEMMED_TOKENS.update(doc.split())
for doc in df['UNSTEMMED_TEXT']:
    UNSTEMMED_TOKENS.update(doc.split())
# create a map of word to token
STEMMED_TOKEN_TO_WORD = {word: i  for i, word in enumerate(STEMMED_TOKENS)}
UNSTEMMED_TOKEN_TO_WORD = {word: i  for i, word in enumerate(UNSTEMMED_TOKENS)}

print("Processing Stemmed Index")
process_batches(df, BATCH_SIZE, stem=True)
print("Processing Unstemmed Index")
process_batches(df, BATCH_SIZE, stem=False)

Processing Stemmed Index
Processing batch 0/ 84
Processing batch 1/ 84
Processing batch 2/ 84
Processing batch 3/ 84
Processing batch 4/ 84
Processing batch 5/ 84
Processing batch 6/ 84
Processing batch 7/ 84
Processing batch 8/ 84
Processing batch 9/ 84
Processing batch 10/ 84
Processing batch 11/ 84
Processing batch 12/ 84
Processing batch 13/ 84
Processing batch 14/ 84
Processing batch 15/ 84
Processing batch 16/ 84
Processing batch 17/ 84
Processing batch 18/ 84
Processing batch 19/ 84
Processing batch 20/ 84
Processing batch 21/ 84
Processing batch 22/ 84
Processing batch 23/ 84
Processing batch 24/ 84
Processing batch 25/ 84
Processing batch 26/ 84
Processing batch 27/ 84
Processing batch 28/ 84
Processing batch 29/ 84
Processing batch 30/ 84
Processing batch 31/ 84
Processing batch 32/ 84
Processing batch 33/ 84
Processing batch 34/ 84
Processing batch 35/ 84
Processing batch 36/ 84
Processing batch 37/ 84
Processing batch 38/ 84
Processing batch 39/ 84
Processing batch 40/ 84
P

## Subtask - Merger

In [46]:
import os
from collections import OrderedDict

def list_files_in_directory(directory):
    """List files in a directory."""
    return [os.path.join(directory, filename) for filename in os.listdir(directory)]

def load_catalog_from_file(filename):
    """Load catalog from a file."""
    catalog = OrderedDict()
    with open(filename, encoding="ISO-8859-1", errors='ignore') as file:
        for line in file:
            token, info = line.rsplit(":", 1)
            start, length = info.split(",")
            catalog[int(token.strip())] = (start.strip(), length.strip())
    return catalog

In [47]:
def merge_indices_and_catalog_without_compression(merge_path, merge_number, catalog1, index1_path, catalog2, index2_path):
    """Merge indices and catalog."""
    merged_catalog_path = f"{merge_path}merged{merge_number}_catalog.txt"
    merged_index_path = f"{merge_path}merged{merge_number}_index.txt"
    
    with open(merged_catalog_path, "x") as merged_catalog_file, open(merged_index_path, "x") as merged_index_file:
        i, j = 0, 0
        offset = 0
        while i < len(catalog1) and j < len(catalog2):
            term1, (start1, length1) = catalog1[i]
            term2, (start2, length2) = catalog2[j]

            if term1 == term2:
                with open(index1_path, "r") as index_file1, open(index2_path, "r") as index_file2:
                    index_file1.seek(int(start1))
                    index_file2.seek(int(start2))
                    info1 = index_file1.read(int(length1))
                    info2 = index_file2.read(int(length2))
                new_info = info1 + info2
                merged_catalog_file.write(f"{term1}:{offset},{len(new_info)}\n")
                merged_index_file.write(new_info)

                i += 1
                j += 1
                offset += len(new_info)
            elif term1 > term2:
                with open(index2_path, "r") as index_file2:
                    index_file2.seek(int(start2))
                    info2 = index_file2.read(int(length2))
                merged_catalog_file.write(f"{term2}:{offset},{len(info2)}\n")
                merged_index_file.write(info2)

                j += 1
                offset += len(info2)
            else:
                with open(index1_path, "r") as index_file1:
                    index_file1.seek(int(start1))
                    info1 = index_file1.read(int(length1))
                merged_catalog_file.write(f"{term1}:{offset},{len(info1)}\n")
                merged_index_file.write(info1)

                i += 1
                offset += len(info1)
        while i < len(catalog1):
            term, (start, length) = catalog1[i]
            with open(index1_path, "r") as index_file1:
                index_file1.seek(int(start))
                info = index_file1.read(int(length))
            merged_catalog_file.write(f"{term}:{offset},{len(info)}\n")
            merged_index_file.write(info)
            
            i += 1
            offset += len(info)
        while j < len(catalog2):
            term, (start, length) = catalog2[j]
            with open(index2_path, "r") as index_file2:
                index_file2.seek(int(start))
                info = index_file2.read(int(length))
            merged_catalog_file.write(f"{term}:{offset},{len(info)}\n")
            merged_index_file.write(info)
        
            j += 1
            offset += len(info)

def master_merger_without_compression(stem=True):
    if stem:
        catalogs_path = STEMMED_CATALOG_PATH
        indices_path = STEMMED_INDEX_PATH
        merging_path = STEMMED_MERGE_PATH
        decompressed_path = STEMMED_DECOMPRESSED_PATH
    else:
        catalogs_path = UNSTEMMED_CATALOG_PATH
        indices_path = UNSTEMMED_INDEX_PATH
        merging_path = UNSTEMMED_MERGE_PATH
        decompressed_path = UNSTEMMED_DECOMPRESSED_PATH

    os.system(f"rm -r {merging_path}*")

    catalog_files = list_files_in_directory(catalogs_path)
    index_files = list_files_in_directory(indices_path)
    catalog_files.sort()
    index_files.sort()

    full_catalog_file = catalog_files[0]
    full_catalog = load_catalog_from_file(full_catalog_file)
    full_catalog_items = list(full_catalog.items())

    full_index_file = index_files[0]

    merge_count = 1

    for i in tqdm(range(1, len(catalog_files))):
        partial_catalog_file = catalog_files[i]
        partial_catalog = load_catalog_from_file(partial_catalog_file)
        partial_catalog_items = list(partial_catalog.items())
        partial_index_file = index_files[i]

        merge_indices_and_catalog_without_compression(merging_path, merge_count, full_catalog_items, full_index_file, partial_catalog_items, partial_index_file)

        full_catalog_file = merging_path + "merged" + str(merge_count) + "_catalog.txt"
        full_catalog = load_catalog_from_file(full_catalog_file)
        full_catalog_items = list(full_catalog.items())
        full_index_file = merging_path + "merged" + str(merge_count) + "_index.txt"

        merge_count += 1

    print("Copied merged files to compressed folder")
    os.system(f"cp {merging_path}merged{merge_count-1}_catalog.txt {decompressed_path}")
    os.system(f"cp {merging_path}merged{merge_count-1}_index.txt {decompressed_path}")

    print("Merging process completed.")

print("Merging stemmed indices...")
master_merger_without_compression(stem=True)
print("Merging unstemmed indices...")
master_merger_without_compression(stem=False)

Merging stemmed indices...


100%|██████████| 84/84 [05:23<00:00,  3.85s/it]


Copied merged files to compressed folder
Merging process completed.
Merging unstemmed indices...


100%|██████████| 84/84 [06:41<00:00,  4.78s/it]

Copied merged files to compressed folder
Merging process completed.


In [52]:
def merge_indices(merge_directory, merge_number, catalog_list_a, index_path_a, catalog_list_b, index_path_b):
    merged_catalog_file = open(merge_directory + "merged" + str(merge_number) + "_catalog.txt", "x")
    merged_index_file = open(merge_directory + "merged" + str(merge_number) + "_index.txt", "wb")
    i, j = 0, 0
    offset = 0

    while i < len(catalog_list_a) and j < len(catalog_list_b):
        term1, (start1, length1) = catalog_list_a[i]
        term2, (start2, length2) = catalog_list_b[j]

        if term1 == term2:
            if merge_number == 1:
                file_a = open(index_path_a, "r")
            else:
                file_a = open(index_path_a, "rb")
            file_b = open(index_path_b, "r")

            file_a.seek(int(start1))
            file_b.seek(int(start2))

            pointer_a_doc_info = file_a.read(int(length1))
            pointer_b_doc_info = file_b.read(int(length2))

            file_a.close()
            file_b.close()

            if merge_number != 1:
                pointer_a_doc_info = zlib.decompress(pointer_a_doc_info)
                pointer_a_doc_info = str(pointer_a_doc_info, 'utf-8')

            new_info = pointer_a_doc_info + pointer_b_doc_info
            new_info_compressed = zlib.compress(new_info.encode('utf-8'), 6)

            merged_catalog_file.write(str(term1) + ":" + str(offset).strip() + "," + str(len(new_info_compressed)) + "\n")
            merged_index_file.write(new_info_compressed)

            i += 1
            j += 1
            offset += len(new_info_compressed)

        elif term1 > term2:

            file_b = open(index_path_b, "r")
            file_b.seek(int(start2))
            pointer_b_doc_info = file_b.read(int(length2))
            file_b.close()

            pointer_b_doc_info_compressed = zlib.compress(pointer_b_doc_info.encode('utf-8'), 6)

            merged_catalog_file.write(str(term2) + ":" + str(offset).strip() + "," + str(len(pointer_b_doc_info_compressed)) + "\n")
            merged_index_file.write(pointer_b_doc_info_compressed)

            j += 1
            offset += len(pointer_b_doc_info_compressed)

        else:

            if merge_number == 1:
                file_a = open(index_path_a, "r")
            else:
                file_a = open(index_path_a, "rb")

            file_a.seek(int(start1))
            pointer_a_doc_info = file_a.read(int(length1))
            file_a.close()

            if merge_number == 1:
                pointer_a_doc_info = zlib.compress(pointer_a_doc_info.encode('utf-8'), 6)

            merged_catalog_file.write(str(term1) + ":" + str(offset).strip() + "," + str(len(pointer_a_doc_info)) + "\n")
            merged_index_file.write(pointer_a_doc_info)

            i += 1
            offset += len(pointer_a_doc_info)

    while i < len(catalog_list_a):
        term, (start, length) = catalog_list_a[i]

        if merge_number == 1:
            file_a = open(index_path_a, "r")
        else:
            file_a = open(index_path_a, "rb")

        file_a.seek(int(start))
        pointer_a_doc_info = file_a.read(int(length))
        file_a.close()

        if merge_number == 1:
            pointer_a_doc_info = zlib.compress(pointer_a_doc_info.encode('utf-8'), 6)

        merged_catalog_file.write(str(term) + ":" + str(offset).strip() + "," + str(len(pointer_a_doc_info)) + "\n")
        merged_index_file.write(pointer_a_doc_info)

        i += 1
        offset += len(pointer_a_doc_info)

    while j < len(catalog_list_b):
        term, (start, length) = catalog_list_b[j]

        file_b = open(index_path_b, "r")
        file_b.seek(int(start))
        pointer_b_doc_info = file_b.read(int(length))
        file_b.close()

        pointer_b_doc_info_compressed = zlib.compress(pointer_b_doc_info.encode('utf-8'), 6)

        merged_catalog_file.write(str(term) + ":" + str(offset).strip() + "," + str(len(pointer_b_doc_info_compressed)) + "\n")
        merged_index_file.write(pointer_b_doc_info_compressed)

        j += 1
        offset += len(pointer_b_doc_info_compressed)

    merged_catalog_file.close()
    merged_index_file.close()

def master_merger(stem=True):
    if stem:
        catalogs_path = STEMMED_CATALOG_PATH
        indices_path = STEMMED_INDEX_PATH
        merging_path = STEMMED_MERGE_PATH
        compressed_path = STEMMED_COMPRESSED_PATH
    else:
        catalogs_path = UNSTEMMED_CATALOG_PATH
        indices_path = UNSTEMMED_INDEX_PATH
        merging_path = UNSTEMMED_MERGE_PATH
        compressed_path = UNSTEMMED_COMPRESSED_PATH
    
    os.system(f"rm -r {merging_path}*")

    catalog_files = list_files_in_directory(catalogs_path)
    index_files = list_files_in_directory(indices_path)
    catalog_files.sort()
    index_files.sort()

    full_catalog_file = catalog_files[0]
    full_catalog = load_catalog_from_file(full_catalog_file)
    full_catalog_items = list(full_catalog.items())

    full_index_file = index_files[0]

    merge_count = 1

    for i in tqdm(range(1, len(catalog_files))):
        partial_catalog_file = catalog_files[i]
        partial_catalog = load_catalog_from_file(partial_catalog_file)
        partial_catalog_items = list(partial_catalog.items())
        partial_index_file = index_files[i]

        merge_indices(merging_path, merge_count, full_catalog_items, full_index_file, partial_catalog_items, partial_index_file)

        full_catalog_file = merging_path + "merged" + str(merge_count) + "_catalog.txt"
        full_catalog = load_catalog_from_file(full_catalog_file)
        full_catalog_items = list(full_catalog.items())
        full_index_file = merging_path + "merged" + str(merge_count) + "_index.txt"

        merge_count += 1

    print("Copied merged files to compressed folder")
    os.system(f"cp {merging_path}merged{merge_count-1}_catalog.txt {compressed_path}")
    os.system(f"cp {merging_path}merged{merge_count-1}_index.txt {compressed_path}")

    print("Merging process completed.")

print("Merging stemmed indices...")
master_merger(stem=True)
print("Merging unstemmed indices...")
master_merger(stem=False)

Merging stemmed indices...


100%|██████████| 84/84 [07:57<00:00,  5.68s/it]


Copied merged files to compressed folder
Merging process completed.
Merging unstemmed indices...


100%|██████████| 84/84 [08:35<00:00,  6.14s/it]

Copied merged files to compressed folder
Merging process completed.


# Task 3 : Searching

In [53]:
def process_queries(stem=True):
    stop_words = set()
    with open(STOP_WORDS_PATH, 'r') as file:
        stop_words = set(file.read().splitlines())
    
    with open(QUERY_PATH, 'r') as file:
        queries = file.readlines()

    queries = {query.split()[0][:-1]: ' '.join(query.split()[1:]).strip() for query in queries}
    if stem:
        queries = {query_no: preprocess_text(query, stop_words, stem=True) for query_no, query in queries.items()}
    else:
        queries = {query_no: preprocess_text(query, stop_words, stem=False) for query_no, query in queries.items()}

    return queries

stemmed_queries = process_queries(stem=True)
unstemmed_queries = process_queries(stem=False)

In [54]:
STEMMED_INVERTED_INDEX_WITH_COMPRESSION = STEMMED_COMPRESSED_PATH + "merged84_index.txt"
UNSTEMMED_INVERTED_INDEX_WITH_COMPRESSION = UNSTEMMED_COMPRESSED_PATH + "merged84_index.txt"

STEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION = STEMMED_DECOMPRESSED_PATH + "merged84_index.txt"
UNSTEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION = UNSTEMMED_DECOMPRESSED_PATH + "merged84_index.txt"

STEMMED_CATALOG_WITH_COMPRESSION = load_catalog_from_file(STEMMED_COMPRESSED_PATH + "merged84_catalog.txt")
UNSTEMMED_CATALOG_WITH_COMPRESSION = load_catalog_from_file(UNSTEMMED_COMPRESSED_PATH + "merged84_catalog.txt")

STEMMED_CATALOG_WITHOUT_COMPRESSION = load_catalog_from_file(STEMMED_DECOMPRESSED_PATH + "merged84_catalog.txt")
UNSTEMMED_CATALOG_WITHOUT_COMPRESSION = load_catalog_from_file(UNSTEMMED_DECOMPRESSED_PATH + "merged84_catalog.txt")

In [55]:
def index_parser(doc_info):
    doc_dict = {}
    for doc in doc_info.strip().split("|")[:-1]:
        doc_no, positions = doc.split(":")
        doc_dict[doc_no] = list(map(int, positions.split(",")))
    return doc_dict

def write_scores(scores_dict, file_name):
    with open(file_name + ".txt", "w") as score_file:
        for i, (query_id, doc_scores) in enumerate(scores_dict.items(),1):
            for doc_id, score in doc_scores.items():
                if i > 1000:
                    break
                line = f"{query_id} Q0 {DOC_NO_MAP[doc_id]} {i+1} {score} Exp\n"
                score_file.write(line)

In [ ]:
# def okapi_score(queries, stem=True, compression=True):
#     if stem:
#         if compression:
#             INVERTED_INDEX = STEMMED_INVERTED_INDEX_WITH_COMPRESSION
#             FULL_CATALOG = STEMMED_CATALOG_WITH_COMPRESSION
#         else:
#             INVERTED_INDEX = STEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION        
#             FULL_CATALOG = STEMMED_CATALOG_WITHOUT_COMPRESSION
#         doc_len_map = DOC_LEN_MAP_STEM
#     else:
#         if compression:
#             INVERTED_INDEX = UNSTEMMED_INVERTED_INDEX_WITH_COMPRESSION
#             FULL_CATALOG = UNSTEMMED_CATALOG_WITH_COMPRESSION
#         else:
#             INVERTED_INDEX = UNSTEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION
#             FULL_CATALOG = UNSTEMMED_CATALOG_WITHOUT_COMPRESSION
#         doc_len_map = DOC_LEN_MAP_UNSTEM

#     file = open(INVERTED_INDEX, "rb" if compression else "r")
        
#     scores = {q_id: {doc: 0 for doc in DOC_NO_MAP.keys()} for q_id, query in queries.items()}
#     for q_id, query in tqdm(queries.items()):
#         for word in query.split():
#             if word in FULL_CATALOG:
#                 start, length = FULL_CATALOG[word]

#                 file.seek(int(start))
#                 doc_info = file.read(int(length))
#                 if compression:
#                     doc_dict = index_parser(str(zlib.decompress(doc_info), 'utf-8'))
#                 else:
#                     doc_dict = index_parser(doc_info)

#                 for doc, positions in doc_dict.items():
#                     tf_wd = len(positions)
#                     curr_doc_len = DOC_LEN_MAP_STEM[int(doc)]
#                     avg_doc_len = sum(DOC_LEN_MAP_STEM.values()) / len(DOC_LEN_MAP_STEM)
#                     scores[q_id][int(doc)] += okapi_tf_formula(tf_wd, curr_doc_len, avg_doc_len)
#         scores[q_id] = {doc: score for doc, score in sorted(scores[q_id].items(), key=lambda item: item[1], reverse=True)}
#     file.close()

#     return scores

# print("OKAPI TF SCORES STEMMED INDEX WITH COMPRESSION")
# okapi_scores = okapi_score(queries, stem=True, compression=True)
# write_scores(okapi_scores, STEMMED_PATH + "results/okapi_scores")
# !perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt stemmed/results/okapi_scores.txt

# print("OKAPI TF SCORES UNSTEMMED INDEX WITH COMPRESSION")
# okapi_scores_us = okapi_score(queries, stem=False, compression=True)
# write_scores(okapi_scores_us, UNSTEMMED_PATH + "results/okapi_scores_us")
# !perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt unstemmed/results/okapi_scores_us.txt

# print("OKAPI TF SCORES STEMMED INDEX WITHOUT COMPRESSION")
# okapi_scores_wc = okapi_score(queries, stem=True, compression=False)
# write_scores(okapi_scores_wc, STEMMED_PATH + "results/okapi_scores_wc")
# !perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt stemmed/results/okapi_scores_wc.txt

# print("OKAPI TF SCORES UNSTEMMED INDEX WITHOUT COMPRESSION")
# okapi_scores_us_wc = okapi_score(queries, stem=False, compression=False)
# write_scores(okapi_scores_us_wc, UNSTEMMED_PATH + "results/okapi_scores_us_wc")
# !perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt unstemmed/results/okapi_scores_us_wc.txt

In [58]:
def okapi_tf_formula(tf_wd, curr_doc_len, avg_doc_len):
    return tf_wd / (tf_wd + 0.5 + (1.5 * (curr_doc_len / avg_doc_len)))

def tf_idf_score(queries, stem=True, compression=True):
    if stem:
        if compression:
            INVERTED_INDEX = STEMMED_INVERTED_INDEX_WITH_COMPRESSION
            FULL_CATALOG = STEMMED_CATALOG_WITH_COMPRESSION
        else:
            INVERTED_INDEX = STEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION        
            FULL_CATALOG = STEMMED_CATALOG_WITHOUT_COMPRESSION
        doc_len_map = DOC_LEN_MAP_STEM
    else:
        if compression:
            INVERTED_INDEX = UNSTEMMED_INVERTED_INDEX_WITH_COMPRESSION
            FULL_CATALOG = UNSTEMMED_CATALOG_WITH_COMPRESSION
        else:
            INVERTED_INDEX = UNSTEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION
            FULL_CATALOG = UNSTEMMED_CATALOG_WITHOUT_COMPRESSION
        doc_len_map = DOC_LEN_MAP_UNSTEM

    file = open(INVERTED_INDEX, "rb" if compression else "r")

    scores = {q_id: {doc: 0 for doc in DOC_NO_MAP.keys()} for q_id, query in queries.items()}
    for q_id, query in tqdm(queries.items()):
        for word in query.split():
            if stem:
                word_key = STEMMED_TOKEN_TO_WORD.get(word, "MISSING")
            else:
                word_key = UNSTEMMED_TOKEN_TO_WORD.get(word, "MISSING")

            if word_key in FULL_CATALOG:
                start, length = FULL_CATALOG[word_key]
                file.seek(int(start))
                doc_info = file.read(int(length))
                if compression:
                    doc_dict = index_parser(str(zlib.decompress(doc_info), 'utf-8'))
                else:
                    doc_dict = index_parser(doc_info)

                for doc, positions in doc_dict.items():
                    tf_wd = len(positions)
                    curr_doc_len = doc_len_map[int(doc)]
                    avg_doc_len = sum(doc_len_map.values()) / len(DOC_LEN_MAP_STEM)
                    doc_freq = len(doc_dict)
                    number_of_documents = len(DOC_NO_MAP)
                    scores[q_id][int(doc)] += okapi_tf_formula(tf_wd, curr_doc_len, avg_doc_len) * math.log(number_of_documents / doc_freq)
        scores[q_id] = {doc: score for doc, score in sorted(scores[q_id].items(), key=lambda item: item[1], reverse=True)}
    file.close()
    return scores

print("TF-IDF SCORES STEMMED INDEX WITH COMPRESSION")
tf_idf_scores = tf_idf_score(stemmed_queries, stem=True, compression=True)
write_scores(tf_idf_scores, STEMMED_PATH + "results/tf_idf_scores")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt stemmed/results/tf_idf_scores.txt

print("TF-IDF SCORES UNSTEMMED INDEX WITH COMPRESSION")
tf_idf_scores_us = tf_idf_score(unstemmed_queries, stem=False, compression=True)
write_scores(tf_idf_scores_us, UNSTEMMED_PATH + "results/tf_idf_scores_us")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt unstemmed/results/tf_idf_scores_us.txt

print("TF-IDF SCORES STEMMED INDEX WITHOUT COMPRESSION")
tf_idf_scores_wc = tf_idf_score(stemmed_queries, stem=True, compression=False)
write_scores(tf_idf_scores_wc, STEMMED_PATH + "results/tf_idf_scores_wc")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt stemmed/results/tf_idf_scores_wc.txt

print("TF-IDF SCORES UNSTEMMED INDEX WITHOUT COMPRESSION")
tf_idf_scores_us_wc = tf_idf_score(unstemmed_queries, stem=False, compression=False)
write_scores(tf_idf_scores_us_wc, UNSTEMMED_PATH + "results/tf_idf_scores_us_wc")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt unstemmed/results/tf_idf_scores_us_wc.txt

TF-IDF SCORES STEMMED INDEX WITH COMPRESSION


100%|██████████| 25/25 [17:10<00:00, 41.21s/it] 


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1194
Interpolated Recall - Precision Averages:
    at 0.00       0.6419
    at 0.10       0.5171
    at 0.20       0.3782
    at 0.30       0.3329
    at 0.40       0.2902
    at 0.50       0.2519
    at 0.60       0.2098
    at 0.70       0.1770
    at 0.80       0.1127
    at 0.90       0.0538
    at 1.00       0.0125
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2570
Precision:
  At    5 docs:   0.4320
  At   10 docs:   0.3960
  At   15 docs:   0.3893
  At   20 docs:   0.3740
  At   30 docs:   0.3520
  At  100 docs:   0.2248
  At  200 docs:   0.1596
  At  500 docs:   0.0830
  At 1000 docs:   0.0478
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2885
TF-IDF SCORES UNSTEMMED INDEX WITH COMPRESSION


100%|██████████| 25/25 [04:26<00:00, 10.65s/it]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1015
Interpolated Recall - Precision Averages:
    at 0.00       0.6619
    at 0.10       0.4840
    at 0.20       0.3058
    at 0.30       0.2621
    at 0.40       0.2259
    at 0.50       0.2002
    at 0.60       0.1660
    at 0.70       0.1340
    at 0.80       0.0784
    at 0.90       0.0394
    at 1.00       0.0083
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2126
Precision:
  At    5 docs:   0.4000
  At   10 docs:   0.3280
  At   15 docs:   0.3120
  At   20 docs:   0.2960
  At   30 docs:   0.2747
  At  100 docs:   0.1896
  At  200 docs:   0.1300
  At  500 docs:   0.0694
  At 1000 docs:   0.0406
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2342
TF-IDF SCORES STEMMED INDEX WITHOUT COMPRESSION


100%|██████████| 25/25 [06:50<00:00, 16.41s/it]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1194
Interpolated Recall - Precision Averages:
    at 0.00       0.6419
    at 0.10       0.5171
    at 0.20       0.3782
    at 0.30       0.3329
    at 0.40       0.2902
    at 0.50       0.2519
    at 0.60       0.2098
    at 0.70       0.1770
    at 0.80       0.1127
    at 0.90       0.0538
    at 1.00       0.0125
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2570
Precision:
  At    5 docs:   0.4320
  At   10 docs:   0.3960
  At   15 docs:   0.3893
  At   20 docs:   0.3740
  At   30 docs:   0.3520
  At  100 docs:   0.2248
  At  200 docs:   0.1596
  At  500 docs:   0.0830
  At 1000 docs:   0.0478
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2885
TF-IDF SCORES UNSTEMMED INDEX WITHOUT COMPRESSION


100%|██████████| 25/25 [04:16<00:00, 10.27s/it]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1015
Interpolated Recall - Precision Averages:
    at 0.00       0.6619
    at 0.10       0.4840
    at 0.20       0.3058
    at 0.30       0.2621
    at 0.40       0.2259
    at 0.50       0.2002
    at 0.60       0.1660
    at 0.70       0.1340
    at 0.80       0.0784
    at 0.90       0.0394
    at 1.00       0.0083
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2126
Precision:
  At    5 docs:   0.4000
  At   10 docs:   0.3280
  At   15 docs:   0.3120
  At   20 docs:   0.2960
  At   30 docs:   0.2747
  At  100 docs:   0.1896
  At  200 docs:   0.1300
  At  500 docs:   0.0694
  At 1000 docs:   0.0406
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2342


In [57]:
def bm25_score(queries, stem=True, compression=True):
    if stem:
        if compression:
            INVERTED_INDEX = STEMMED_INVERTED_INDEX_WITH_COMPRESSION
            FULL_CATALOG = STEMMED_CATALOG_WITH_COMPRESSION
        else:
            INVERTED_INDEX = STEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION        
            FULL_CATALOG = STEMMED_CATALOG_WITHOUT_COMPRESSION
        doc_len_map = DOC_LEN_MAP_STEM
    else:
        if compression:
            INVERTED_INDEX = UNSTEMMED_INVERTED_INDEX_WITH_COMPRESSION
            FULL_CATALOG = UNSTEMMED_CATALOG_WITH_COMPRESSION
        else:
            INVERTED_INDEX = UNSTEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION
            FULL_CATALOG = UNSTEMMED_CATALOG_WITHOUT_COMPRESSION
        doc_len_map = DOC_LEN_MAP_UNSTEM
    
    file = open(INVERTED_INDEX, "rb" if compression else "r")

    scores = {q_id: {doc: 0 for doc in DOC_NO_MAP.keys()} for q_id, query in queries.items()}
    k1 = 1.2
    b = 0.75
    for q_id, query in tqdm(queries.items()):
        for word in query.split():
            if stem:
                word_key = STEMMED_TOKEN_TO_WORD.get(word, "MISSING")
            else:
                word_key = UNSTEMMED_TOKEN_TO_WORD.get(word, "MISSING")

            if word_key in FULL_CATALOG:
                start, length = FULL_CATALOG[word_key]
                file.seek(int(start))
                doc_info = file.read(int(length))
                if compression:
                    doc_dict = index_parser(str(zlib.decompress(doc_info), 'utf-8'))
                else:
                    doc_dict = index_parser(doc_info)

                for doc, positions in doc_dict.items():
                    tf_wd = len(positions)
                    curr_doc_len = doc_len_map[int(doc)]
                    avg_doc_len = sum(doc_len_map.values()) / len(doc_len_map)
                    doc_freq = len(doc_dict)
                    number_of_documents = len(DOC_NO_MAP)
                    idf = math.log((number_of_documents - doc_freq + 0.5) / (doc_freq + 0.5))
                    bm25 = idf * ((tf_wd * (k1 + 1)) / (tf_wd + k1 * (1 - b + b * (curr_doc_len / avg_doc_len))))
                    scores[q_id][int(doc)] += bm25
        scores[q_id] = {doc: score for doc, score in sorted(scores[q_id].items(), key=lambda item: item[1], reverse=True)}
    file.close()
    return scores

print("BM25 SCORES STEMMED INDEX WITH COMPRESSION")
bm25_scores = bm25_score(stemmed_queries, stem=True, compression=True)
write_scores(bm25_scores, STEMMED_PATH + "results/bm25_scores")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt stemmed/results/bm25_scores.txt

print("BM25 SCORES UNSTEMMED INDEX WITH COMPRESSION")
bm25_scores_us = bm25_score(unstemmed_queries, stem=False, compression=True)
write_scores(bm25_scores_us, UNSTEMMED_PATH + "results/bm25_scores_us")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt unstemmed/results/bm25_scores_us.txt

print("BM25 SCORES STEMMED INDEX WITHOUT COMPRESSION")
bm25_scores_wc = bm25_score(stemmed_queries, stem=True, compression=False)
write_scores(bm25_scores_wc, STEMMED_PATH + "results/bm25_scores_wc")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt stemmed/results/bm25_scores_wc.txt

print("BM25 SCORES UNSTEMMED INDEX WITHOUT COMPRESSION")
bm25_scores_us_wc = bm25_score(unstemmed_queries, stem=False, compression=False)
write_scores(bm25_scores_us_wc, UNSTEMMED_PATH + "results/bm25_scores_us_wc")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt unstemmed/results/bm25_scores_us_wc.txt

BM25 SCORES STEMMED INDEX WITH COMPRESSION


100%|██████████| 25/25 [07:29<00:00, 17.97s/it]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1197
Interpolated Recall - Precision Averages:
    at 0.00       0.6863
    at 0.10       0.5343
    at 0.20       0.4075
    at 0.30       0.3408
    at 0.40       0.2944
    at 0.50       0.2572
    at 0.60       0.2117
    at 0.70       0.1873
    at 0.80       0.1323
    at 0.90       0.0517
    at 1.00       0.0139
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2665
Precision:
  At    5 docs:   0.4480
  At   10 docs:   0.4040
  At   15 docs:   0.3867
  At   20 docs:   0.3840
  At   30 docs:   0.3600
  At  100 docs:   0.2340
  At  200 docs:   0.1608
  At  500 docs:   0.0848
  At 1000 docs:   0.0479
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2952
BM25 SCORES UNSTEMMED INDEX WITH COMPRESSION


100%|██████████| 25/25 [04:36<00:00, 11.05s/it]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1037
Interpolated Recall - Precision Averages:
    at 0.00       0.7079
    at 0.10       0.4879
    at 0.20       0.3187
    at 0.30       0.2593
    at 0.40       0.2358
    at 0.50       0.2089
    at 0.60       0.1764
    at 0.70       0.1446
    at 0.80       0.0965
    at 0.90       0.0570
    at 1.00       0.0124
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2214
Precision:
  At    5 docs:   0.4000
  At   10 docs:   0.3520
  At   15 docs:   0.3200
  At   20 docs:   0.3000
  At   30 docs:   0.2827
  At  100 docs:   0.1928
  At  200 docs:   0.1334
  At  500 docs:   0.0713
  At 1000 docs:   0.0415
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2464
BM25 SCORES STEMMED INDEX WITHOUT COMPRESSION


100%|██████████| 25/25 [07:06<00:00, 17.06s/it]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1197
Interpolated Recall - Precision Averages:
    at 0.00       0.6863
    at 0.10       0.5343
    at 0.20       0.4075
    at 0.30       0.3408
    at 0.40       0.2944
    at 0.50       0.2572
    at 0.60       0.2117
    at 0.70       0.1873
    at 0.80       0.1323
    at 0.90       0.0517
    at 1.00       0.0139
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2665
Precision:
  At    5 docs:   0.4480
  At   10 docs:   0.4040
  At   15 docs:   0.3867
  At   20 docs:   0.3840
  At   30 docs:   0.3600
  At  100 docs:   0.2340
  At  200 docs:   0.1608
  At  500 docs:   0.0848
  At 1000 docs:   0.0479
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2952
BM25 SCORES UNSTEMMED INDEX WITHOUT COMPRESSION


100%|██████████| 25/25 [04:36<00:00, 11.06s/it]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1037
Interpolated Recall - Precision Averages:
    at 0.00       0.7079
    at 0.10       0.4879
    at 0.20       0.3187
    at 0.30       0.2593
    at 0.40       0.2358
    at 0.50       0.2089
    at 0.60       0.1764
    at 0.70       0.1446
    at 0.80       0.0965
    at 0.90       0.0570
    at 1.00       0.0124
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2214
Precision:
  At    5 docs:   0.4000
  At   10 docs:   0.3520
  At   15 docs:   0.3200
  At   20 docs:   0.3000
  At   30 docs:   0.2827
  At  100 docs:   0.1928
  At  200 docs:   0.1334
  At  500 docs:   0.0713
  At 1000 docs:   0.0415
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2464


In [59]:
def lm_laplace_smooth_score(queries, stem=True, compression=True):
    if stem:
        if compression:
            INVERTED_INDEX = STEMMED_INVERTED_INDEX_WITH_COMPRESSION
            FULL_CATALOG = STEMMED_CATALOG_WITH_COMPRESSION
        else:
            INVERTED_INDEX = STEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION        
            FULL_CATALOG = STEMMED_CATALOG_WITHOUT_COMPRESSION
        doc_len_map = DOC_LEN_MAP_STEM
    else:
        if compression:
            INVERTED_INDEX = UNSTEMMED_INVERTED_INDEX_WITH_COMPRESSION
            FULL_CATALOG = UNSTEMMED_CATALOG_WITH_COMPRESSION
        else:
            INVERTED_INDEX = UNSTEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION
            FULL_CATALOG = UNSTEMMED_CATALOG_WITHOUT_COMPRESSION
        doc_len_map = DOC_LEN_MAP_UNSTEM
    
    file = open(INVERTED_INDEX, "rb" if compression else "r")

    scores = {q_id: {doc: 0 for doc in DOC_NO_MAP.keys()} for q_id, query in queries.items()}
    # Find the vocabulary size
    V = len(FULL_CATALOG)
    for q_id, query in tqdm(queries.items()):
        for word in query.split():
            if stem:
                word_key = STEMMED_TOKEN_TO_WORD.get(word, "MISSING")
            else:
                word_key = UNSTEMMED_TOKEN_TO_WORD.get(word, "MISSING")

            if word_key in FULL_CATALOG:            
                start, length = FULL_CATALOG[word_key]
                file.seek(int(start))
                doc_info = file.read(int(length))
                if compression:
                    doc_dict = index_parser(str(zlib.decompress(doc_info), 'utf-8'))
                else:
                    doc_dict = index_parser(doc_info)
                
                for doc, positions in doc_dict.items():
                    tf_wd = len(positions)
                    curr_doc_len = doc_len_map[int(doc)]
                    scores[q_id][int(doc)] += np.log((tf_wd + 1) / (curr_doc_len + V))
                
                for doc in DOC_NO_MAP.keys():
                    if str(doc) not in doc_dict.keys():
                        scores[q_id][doc] += -100

        scores[q_id] = {doc: score for doc, score in sorted(scores[q_id].items(), key=lambda item: item[1], reverse=True)}
    file.close()
    return scores

print("LM LAPLACE SMOOTH SCORES STEMMED INDEX WITH COMPRESSION")
lm_laplace_scores = lm_laplace_smooth_score(stemmed_queries, stem=True, compression=True)
write_scores(lm_laplace_scores, STEMMED_PATH + "results/lm_laplace_scores")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt stemmed/results/lm_laplace_scores.txt

print("LM LAPLACE SMOOTH SCORES UNSTEMMED INDEX WITH COMPRESSION")
lm_laplace_scores_us = lm_laplace_smooth_score(unstemmed_queries, stem=False, compression=True)
write_scores(lm_laplace_scores_us, UNSTEMMED_PATH + "results/lm_laplace_scores_us")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt unstemmed/results/lm_laplace_scores_us.txt

print("LM LAPLACE SMOOTH SCORES STEMMED INDEX WITHOUT COMPRESSION")
lm_laplace_scores_wc = lm_laplace_smooth_score(stemmed_queries, stem=True, compression=False)
write_scores(lm_laplace_scores_wc, STEMMED_PATH + "results/lm_laplace_scores_wc")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt stemmed/results/lm_laplace_scores_wc.txt

print("LM LAPLACE SMOOTH SCORES UNSTEMMED INDEX WITHOUT COMPRESSION")
lm_laplace_scores_us_wc = lm_laplace_smooth_score(unstemmed_queries, stem=False, compression=False)
write_scores(lm_laplace_scores_us_wc, UNSTEMMED_PATH + "results/lm_laplace_scores_us_wc")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt unstemmed/results/lm_laplace_scores_us_wc.txt

LM LAPLACE SMOOTH SCORES STEMMED INDEX WITH COMPRESSION


100%|██████████| 25/25 [00:05<00:00,  4.29it/s]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1094
Interpolated Recall - Precision Averages:
    at 0.00       0.6153
    at 0.10       0.4866
    at 0.20       0.4010
    at 0.30       0.3156
    at 0.40       0.2414
    at 0.50       0.1898
    at 0.60       0.1450
    at 0.70       0.1086
    at 0.80       0.0665
    at 0.90       0.0258
    at 1.00       0.0000
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2180
Precision:
  At    5 docs:   0.4240
  At   10 docs:   0.4080
  At   15 docs:   0.3680
  At   20 docs:   0.3420
  At   30 docs:   0.3093
  At  100 docs:   0.2076
  At  200 docs:   0.1438
  At  500 docs:   0.0744
  At 1000 docs:   0.0438
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2613
LM LAPLACE SMOOTH SCORES UNSTEMMED INDEX WITH COMPRESSION


100%|██████████| 25/25 [00:04<00:00,  5.13it/s]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1003
Interpolated Recall - Precision Averages:
    at 0.00       0.6478
    at 0.10       0.4441
    at 0.20       0.3376
    at 0.30       0.2787
    at 0.40       0.2143
    at 0.50       0.1811
    at 0.60       0.1328
    at 0.70       0.0993
    at 0.80       0.0689
    at 0.90       0.0298
    at 1.00       0.0000
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.1995
Precision:
  At    5 docs:   0.3600
  At   10 docs:   0.3480
  At   15 docs:   0.3227
  At   20 docs:   0.3120
  At   30 docs:   0.2787
  At  100 docs:   0.1900
  At  200 docs:   0.1290
  At  500 docs:   0.0680
  At 1000 docs:   0.0401
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2446
LM LAPLACE SMOOTH SCORES STEMMED INDEX WITHOUT COMPRESSION


100%|██████████| 25/25 [00:05<00:00,  4.40it/s]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1094
Interpolated Recall - Precision Averages:
    at 0.00       0.6153
    at 0.10       0.4866
    at 0.20       0.4010
    at 0.30       0.3156
    at 0.40       0.2414
    at 0.50       0.1898
    at 0.60       0.1450
    at 0.70       0.1086
    at 0.80       0.0665
    at 0.90       0.0258
    at 1.00       0.0000
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2180
Precision:
  At    5 docs:   0.4240
  At   10 docs:   0.4080
  At   15 docs:   0.3680
  At   20 docs:   0.3420
  At   30 docs:   0.3093
  At  100 docs:   0.2076
  At  200 docs:   0.1438
  At  500 docs:   0.0744
  At 1000 docs:   0.0438
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2613
LM LAPLACE SMOOTH SCORES UNSTEMMED INDEX WITHOUT COMPRESSION


100%|██████████| 25/25 [00:04<00:00,  5.06it/s]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1003
Interpolated Recall - Precision Averages:
    at 0.00       0.6478
    at 0.10       0.4441
    at 0.20       0.3376
    at 0.30       0.2787
    at 0.40       0.2143
    at 0.50       0.1811
    at 0.60       0.1328
    at 0.70       0.0993
    at 0.80       0.0689
    at 0.90       0.0298
    at 1.00       0.0000
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.1995
Precision:
  At    5 docs:   0.3600
  At   10 docs:   0.3480
  At   15 docs:   0.3227
  At   20 docs:   0.3120
  At   30 docs:   0.2787
  At  100 docs:   0.1900
  At  200 docs:   0.1290
  At  500 docs:   0.0680
  At 1000 docs:   0.0401
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2446


In [92]:
def calculate_accumulator(total_no_of_docs, proximity_threshold, word, word_positions_dict):
    current_positions = word_positions_dict.get(word)
    accumulator_value = 0
    for term, positions in word_positions_dict.items():
        if term == word:
            continue
        term_positions = 0
        current_term_positions = 0
        term_j_positions = positions
        while term_positions < len(current_positions) and current_term_positions < len(term_j_positions):
            window = abs(int(current_positions[term_positions]) - int(term_j_positions[current_term_positions]))
            if window <= proximity_threshold:
                term_frequency = len(term_j_positions)
                temp_accumulator = (math.log((total_no_of_docs) / (term_frequency + 1), 2)) / proximity_threshold ** 2
                accumulator_value += temp_accumulator
            if int(current_positions[term_positions]) < int(term_j_positions[current_term_positions]):
                term_positions += 1
            else:
                current_term_positions += 1
        while term_positions < len(current_positions):
            window = abs(int(current_positions[term_positions]) - int(term_j_positions[current_term_positions - 1]))
            if window <= proximity_threshold:
                term_frequency = len(term_j_positions)
                temp_accumulator = (math.log((total_no_of_docs) / (term_frequency + 1), 2)) / proximity_threshold ** 2
                accumulator_value += temp_accumulator
            term_positions += 1
        while current_term_positions < len(term_j_positions):
            window = abs(int(current_positions[term_positions - 1]) - int(term_j_positions[current_term_positions]))
            if window <= proximity_threshold:
                term_frequency = len(term_j_positions)
                temp_accumulator = (math.log((total_no_of_docs) / (term_frequency + 1), 2)) / proximity_threshold ** 2
                accumulator_value += temp_accumulator
            current_term_positions += 1
    return accumulator_value

def calculate_proximity_score(no_of_docs, term_frequency, accumulator_term, document_length, average_document_length, k1, b):
    inverse_document_frequency = math.log((no_of_docs) / (term_frequency + 1), 2)
    calculation_2 = (accumulator_term * (k1 + 1)) / (accumulator_term + (k1 * ((1 - b) + (b * (document_length / average_document_length)))))
    proximity_score = min(inverse_document_frequency, 1) * calculation_2
    return proximity_score

def proximity_search(query_dictionary, stem=True, compression=True):
    if stem:
        if compression:
            INVERTED_INDEX = STEMMED_INVERTED_INDEX_WITH_COMPRESSION
            FULL_CATALOG = STEMMED_CATALOG_WITH_COMPRESSION
        else:
            INVERTED_INDEX = STEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION        
            FULL_CATALOG = STEMMED_CATALOG_WITHOUT_COMPRESSION
        doc_len_map = DOC_LEN_MAP_STEM
    else:
        if compression:
            INVERTED_INDEX = UNSTEMMED_INVERTED_INDEX_WITH_COMPRESSION
            FULL_CATALOG = UNSTEMMED_CATALOG_WITH_COMPRESSION
        else:
            INVERTED_INDEX = UNSTEMMED_INVERTED_INDEX_WITHOUT_COMPRESSION
            FULL_CATALOG = UNSTEMMED_CATALOG_WITHOUT_COMPRESSION
        doc_len_map = DOC_LEN_MAP_UNSTEM
      
    file = open(INVERTED_INDEX, "rb" if compression else "r")
      
    k1 = 1.5
    b = 0.4
    proximity_threshold = 2
    average_document_length = sum(doc_len_map.values()) / len(doc_len_map)
    scores = bm25_score(query_dictionary, doc_len_map, stem)
    for query_id, query_terms in tqdm(query_dictionary.items()):
        query_info = {}
        for term in query_terms.split():
            if stem:
                word_key = STEMMED_TOKEN_TO_WORD.get(term, "MISSING")
            else:
                word_key = UNSTEMMED_TOKEN_TO_WORD.get(term, "MISSING")
            term = word_key

            index_placement = FULL_CATALOG.get(word_key, 0)
            if index_placement != 0:
                offset = index_placement[0]
                length = index_placement[1]
                file.seek(int(offset))
                document_details = file.read(int(length))
                if compression:
                    document_details = index_parser(zlib.decompress(document_details).decode('utf-8'))
                else:
                    document_details = index_parser(document_details)

                for doc_id, positions in document_details.items():
                    if doc_id in query_info:
                        query_info[doc_id][term] = positions
                    else:
                        query_info[doc_id] = {term: positions}

        for doc_id, terms_positions in query_info.items():
            for term, positions in terms_positions.items():
                term_frequency = len(positions)
                current_accumulator = calculate_accumulator(len(DOC_NO_MAP), proximity_threshold, term, terms_positions)
                proximity_score = calculate_proximity_score(len(DOC_NO_MAP), term_frequency, current_accumulator, int(doc_id), average_document_length, k1, b)

                scores[query_id][int(doc_id)] += proximity_score
    file.close()
    return scores

print("PROXIMITY SEARCH SCORES STEMMED INDEX WITH COMPRESSION")
proximity_search_scores = proximity_search(stemmed_queries, stem=True, compression=True)
write_scores(proximity_search_scores, STEMMED_PATH + "results/proximity_search_scores")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt stemmed/results/proximity_search_scores.txt

print("PROXIMITY SEARCH SCORES UNSTEMMED INDEX WITH COMPRESSION")
proximity_search_scores_us = proximity_search(unstemmed_queries, stem=False, compression=True)
write_scores(proximity_search_scores_us, UNSTEMMED_PATH + "results/proximity_search_scores_us")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt unstemmed/results/proximity_search_scores_us.txt

print("PROXIMITY SEARCH SCORES STEMMED INDEX WITHOUT COMPRESSION")
proximity_search_scores_wc = proximity_search(stemmed_queries, stem=True, compression=False)
write_scores(proximity_search_scores_wc, STEMMED_PATH + "results/proximity_search_scores_wc")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt stemmed/results/proximity_search_scores_wc.txt

print("PROXIMITY SEARCH SCORES UNSTEMMED INDEX WITHOUT COMPRESSION")
proximity_search_scores_us_wc = proximity_search(unstemmed_queries, stem=False, compression=False)
write_scores(proximity_search_scores_us_wc, UNSTEMMED_PATH + "results/proximity_search_scores_us_wc")
!perl ../trec_eval/trec_eval.pl ../AP_DATA/qrels.adhoc.51-100.AP89.txt unstemmed/results/proximity_search_scores_us_wc.txt

PROXIMITY SEARCH SCORES STEMMED INDEX WITH COMPRESSION


100%|██████████| 25/25 [00:06<00:00,  4.13it/s]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1198
Interpolated Recall - Precision Averages:
    at 0.00       0.7169
    at 0.10       0.5059
    at 0.20       0.4108
    at 0.30       0.3504
    at 0.40       0.2926
    at 0.50       0.2580
    at 0.60       0.2154
    at 0.70       0.1883
    at 0.80       0.1318
    at 0.90       0.0544
    at 1.00       0.0137
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2673
Precision:
  At    5 docs:   0.4640
  At   10 docs:   0.4200
  At   15 docs:   0.4053
  At   20 docs:   0.3760
  At   30 docs:   0.3493
  At  100 docs:   0.2312
  At  200 docs:   0.1612
  At  500 docs:   0.0853
  At 1000 docs:   0.0479
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2970
PROXIMITY SEARCH SCORES UNSTEMMED INDEX WITH COMPRESSION


100%|██████████| 25/25 [00:03<00:00,  6.38it/s]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1087
Interpolated Recall - Precision Averages:
    at 0.00       0.4632
    at 0.10       0.3149
    at 0.20       0.2715
    at 0.30       0.2343
    at 0.40       0.1889
    at 0.50       0.1601
    at 0.60       0.1265
    at 0.70       0.1066
    at 0.80       0.0769
    at 0.90       0.0526
    at 1.00       0.0136
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.1698
Precision:
  At    5 docs:   0.3360
  At   10 docs:   0.2800
  At   15 docs:   0.2667
  At   20 docs:   0.2500
  At   30 docs:   0.2453
  At  100 docs:   0.1712
  At  200 docs:   0.1242
  At  500 docs:   0.0716
  At 1000 docs:   0.0435
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.1967
PROXIMITY SEARCH SCORES STEMMED INDEX WITHOUT COMPRESSION


100%|██████████| 25/25 [00:05<00:00,  4.38it/s]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1198
Interpolated Recall - Precision Averages:
    at 0.00       0.7169
    at 0.10       0.5059
    at 0.20       0.4108
    at 0.30       0.3504
    at 0.40       0.2926
    at 0.50       0.2580
    at 0.60       0.2154
    at 0.70       0.1883
    at 0.80       0.1318
    at 0.90       0.0544
    at 1.00       0.0137
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.2673
Precision:
  At    5 docs:   0.4640
  At   10 docs:   0.4200
  At   15 docs:   0.4053
  At   20 docs:   0.3760
  At   30 docs:   0.3493
  At  100 docs:   0.2312
  At  200 docs:   0.1612
  At  500 docs:   0.0853
  At 1000 docs:   0.0479
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.2970
PROXIMITY SEARCH SCORES UNSTEMMED INDEX WITHOUT COMPRESSION


100%|██████████| 25/25 [00:03<00:00,  6.98it/s]


Error due to 25

Queryid (Num):       25
Total number of documents over all queries
    Retrieved:    25000
    Relevant:      1832
    Rel_ret:       1087
Interpolated Recall - Precision Averages:
    at 0.00       0.4632
    at 0.10       0.3149
    at 0.20       0.2715
    at 0.30       0.2343
    at 0.40       0.1889
    at 0.50       0.1601
    at 0.60       0.1265
    at 0.70       0.1066
    at 0.80       0.0769
    at 0.90       0.0526
    at 1.00       0.0136
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.1698
Precision:
  At    5 docs:   0.3360
  At   10 docs:   0.2800
  At   15 docs:   0.2667
  At   20 docs:   0.2500
  At   30 docs:   0.2453
  At  100 docs:   0.1712
  At  200 docs:   0.1242
  At  500 docs:   0.0716
  At 1000 docs:   0.0435
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.1967
